In [49]:
merge_functional = False # Whether to merge Functional comments into one class
n_gram = 1  # here we chose [1, 2, 3] for [unigrams, bigrams, trigrams]
l_penalty = 'l2' # we can choose between l1 and l2 for logistic regression and SVM
param_min_df = 1
param_max_df = 1.0
use_stemmer = True
use_tf_idf = True
use_idf = False
remove_special_chars = True
use_lowercase = True

In [10]:
import pandas as pd 
import numpy as np
import re
from time import time

# t_start = time()

In [11]:
import warnings
warnings.filterwarnings('ignore')

In [8]:
def create_common_df():        
        df = pd.read_csv('C:\\Users\\ASUS\\Desktop\\ETF\\V\\OPJ\\OPJ_projekat\\FinalData\\MergedCommonFull.txt', sep='\t', lineterminator='\r')
        df = df.drop(['NaturalLanguageID', 'ProgrammingLanguageName', 'RepoID', 'SourceID', 'CommentID'],axis=1)
        return df

In [9]:
df = create_common_df()

FileNotFoundError: [Errno 2] File b'C:\\Users\\ASUS\\Desktop\\ETF\\V\\OPJ\\OPJ_projekat\\FinalData\\MergedCommonFull.txt' does not exist: b'C:\\Users\\ASUS\\Desktop\\ETF\\V\\OPJ\\OPJ_projekat\\FinalData\\MergedCommonFull.txt'

In [6]:
df

,comment,label_1,label_2,label_3,label_4
0,sets status tp true or false,None,Functional-Method,Functional-Method,Functional-Method
1,Room numbers - Halim \n ********************...,None,Functional-Inline,Functional-Inline,Functional-Inline
2,label \n,None,Functional-Inline,Functional-Inline,Functional-Inline
3,klik na dugme guest na pocetku programa \n,Functional-Inline,Functional-Inline,Functional-Inline,Functional-Inline
4,prikaz slobodnih soba za izabrani tip priliko...,Functional-Inline,Functional-Inline,Functional-Inline,Functional-Method
...,...,...,...,...,...
656,\n * Rečnik koji se koristi za normalizovanje...,Functional-Inline,Functional-Inline,Functional-Inline,Functional-Inline
657,Očuvava kapitalizaciju na početku rečenice \n...,Functional-Inline,Functional-Inline,Functional-Inline,Functional-Inline
658,\n * Ako se radi o karakteru koje spada u eng...,Functional-Inline,Functional-Inline,Functional-Inline,Functional-Inline
659,\n * Dužina (u karakterima) najdužeg sufiksno...,Functional-Inline,Functional-Inline,Functional-Inline,Functional-Inline


In [7]:
pd.set_option('display.max_rows', 10)

In [8]:
df.drop_duplicates(subset ="comment", keep = False, inplace = True)
df = df.dropna()
df = df.reset_index(drop=True)
df

,comment,label_1,label_2,label_3,label_4
0,sets status tp true or false,None,Functional-Method,Functional-Method,Functional-Method
1,Room numbers - Halim \n ********************...,None,Functional-Inline,Functional-Inline,Functional-Inline
2,label \n,None,Functional-Inline,Functional-Inline,Functional-Inline
3,klik na dugme guest na pocetku programa \n,Functional-Inline,Functional-Inline,Functional-Inline,Functional-Inline
4,prikaz slobodnih soba za izabrani tip priliko...,Functional-Inline,Functional-Inline,Functional-Inline,Functional-Method
...,...,...,...,...,...
555,"transformations.put(""naca"", ""nca""); // Batan...",None,None,None,None
556,\n * Rečnik koji se koristi za normalizovanje...,Functional-Inline,Functional-Inline,Functional-Inline,Functional-Inline
557,Očuvava kapitalizaciju na početku rečenice \n...,Functional-Inline,Functional-Inline,Functional-Inline,Functional-Inline
558,\n * Ako se radi o karakteru koje spada u eng...,Functional-Inline,Functional-Inline,Functional-Inline,Functional-Inline


In [9]:
hits = np.zeros((df.shape[0],6))

for i in range(df.shape[0]):
    if df.loc[i]['label_1'] == df.loc[i]['label_2']:
        hits[i][0] += 1
    if df.loc[i]['label_1'] == df.loc[i]['label_3']:
        hits[i][1] += 1
    if df.loc[i]['label_1'] == df.loc[i]['label_4']:
        hits[i][2] += 1
    if df.loc[i]['label_2'] == df.loc[i]['label_3']:
        hits[i][3] += 1
    if df.loc[i]['label_2'] == df.loc[i]['label_4']:
        hits[i][4] += 1
    if df.loc[i]['label_3'] == df.loc[i]['label_4']:
        hits[i][5] += 1

In [10]:
all_same = 0
three_same_one_diff = 0
two_same_two_diff = 0
two_same_one_diff_one_diff = 0
all_diff = 0

for i in range(hits.shape[0]):
    if hits[i].sum() == 6:
        all_same += 1 
    if hits[i].sum() == 3:
        three_same_one_diff += 1
    if hits[i].sum() == 2:
        two_same_two_diff += 1
        
    if hits[i].sum() == 1:
        two_same_one_diff_one_diff += 1
    if hits[i].sum() == 0:
        all_diff += 1

In [11]:
print('All same: ', all_same)
print('Three same, one different: ', three_same_one_diff)
print('Two same, two different (same to one another): ', two_same_two_diff)
print('Two same, two different (different from one another): ', two_same_one_diff_one_diff)
print('All different: ', all_diff)
print('Total comments: ', df.shape[0])

All same:  362
Three same, one different:  112
Two same, two different (same to one another):  45
Two same, two different (different from one another):  32
All different:  9
Total comments:  560


In [12]:
print('All same (%): ', round(all_same/df.shape[0]*100))
print('Three same, one different (%): ',  round(three_same_one_diff/df.shape[0]*100))
print('Two same, two different (same to one another) (%): ',  round(two_same_two_diff/df.shape[0]*100))
print('Two same, two different (different from one another) (%): ',  round(two_same_one_diff_one_diff/df.shape[0]*100))
print('All different (%): ',  round(all_diff/df.shape[0]*100))

All same (%):  65
Three same, one different (%):  20
Two same, two different (same to one another) (%):  8
Two same, two different (different from one another) (%):  6
All different (%):  2


In [13]:
print('Procenat podudarnosti izmedju Laze i Matije: ', round(hits[:,0].sum()/df.shape[0]*100))
print('Procenat podudarnosti izmedju Laze i Mateje: ', round(hits[:,1].sum()/df.shape[0]*100))
print('Procenat podudarnosti izmedju Laze i Munje: ', round(hits[:,2].sum()/df.shape[0]*100))
print('Procenat podudarnosti izmedju Matije i Mateje: ', round(hits[:,3].sum()/df.shape[0]*100))
print('Procenat podudarnosti izmedju Matije i Ivane: ', round(hits[:,4].sum()/df.shape[0]*100))
print('Procenat podudarnosti izmedju Mateje i Ivane: ', round(hits[:,5].sum()/df.shape[0]*100))

Procenat podudarnosti izmedju Laze i Matije:  79.0
Procenat podudarnosti izmedju Laze i Mateje:  84.0
Procenat podudarnosti izmedju Laze i Munje:  79.0
Procenat podudarnosti izmedju Matije i Mateje:  85.0
Procenat podudarnosti izmedju Matije i Ivane:  70.0
Procenat podudarnosti izmedju Mateje i Ivane:  72.0


## Ucitavanje svih podataka

In [1]:
import pandas as pd
def create_df():
    dfs = pd.DataFrame()
    for id_ in range(1,5):
        try:
            cols = ['NaturalLanguageID', 'ProgrammingLanguageName', 'RepoID', 'SourceID', 'CommentID', 'comment', 'label']
            df = pd.read_csv('.\\AnnotatedData\\OutData{}.txt'.format(id_), sep='\t', lineterminator='\r', header=None)
            df.columns = cols
            if id_ == 1:
                dfs = df
            else:
                dfs = pd.concat([dfs,df], ignore_index=True)
        
        except:
            with open('.\\AnnotatedData\\OutData{}.txt'.format(id_), "r", encoding="utf-8",  newline="\r\n") as source:
                for line in source:
                    line = line.replace("\n", "")
                    part = line.split("\t")
                    if len(part) != 7:
                        print('POPRAVITI u fajlu: ', id_)
                        print(part)
    return dfs

In [2]:
df = create_df()

In [3]:
df

,NaturalLanguageID,ProgrammingLanguageName,RepoID,SourceID,CommentID,comment,label
0,SR,Java,BILD-IT-Advanced-master,src\grupniProjekat_02_03_2016\SearchEngine.java,BILD-IT-Advanced-master/src\grupniProjekat_02_...,Ispisujemo poruku da je drajver loadovan,Functional-Inline
1,\nSR,Java,BILD-IT-Advanced-master,src\grupniProjekat_02_03_2016\SearchEngine.java,BILD-IT-Advanced-master/src\grupniProjekat_02_...,Ispis država za uneseni kontinent,Functional-Inline
2,\nSR,Java,BILD-IT-Advanced-master,src\grupniProjekat_02_03_2016\SearchEngine.java,BILD-IT-Advanced-master/src\grupniProjekat_02_...,Slanje upute,Functional-Inline
3,\nSR,Java,BILD-IT-Advanced-master,src\grupniProjekat_02_03_2016\SearchEngine.java,BILD-IT-Advanced-master/src\grupniProjekat_02_...,Zatvaranje konekcije sa bazom podataka,Functional-Inline
4,\nSR,Java,BILD-IT-Advanced-master,src\grupniProjekat_HotelManagement\Hotel.java,BILD-IT-Advanced-master/src\grupniProjekat_Hot...,vraca program na pocetak,Functional-Inline
...,...,...,...,...,...,...,...
3371,\nEN/SR,Java,SCStemmers-master,src\weka\core\stemmers\SCStemmer.java,SCStemmers-master/src\weka\core\stemmers\SCSte...,* Osnovna apstraktna klasa za funkcije zajed...,Functional-Module
3372,\nSR,Java,SCStemmers-master,src\weka\core\stemmers\SCStemmer.java,SCStemmers-master/src\weka\core\stemmers\SCSte...,* Inicijalizuje pravila za stemovanje * <p>...,Functional-Method
3373,\nSR,Java,SCStemmers-master,src\weka\core\stemmers\SerbianStemmer.java,SCStemmers-master/src\weka\core\stemmers\Serbi...,* Spisak sufiksnih pravila * <p> * <i>The ...,Functional-Inline
3374,\nSR,Java,SCStemmers-master,src\weka\core\stemmers\SerbianStemmer.java,SCStemmers-master/src\weka\core\stemmers\Serbi...,* Stemuje reč koja je napisana u dual1 kodir...,Functional-Method


In [4]:
for i in range(df.shape[0]):
    if df['NaturalLanguageID'][i][0] == '\n':
        df['NaturalLanguageID'][i] = df['NaturalLanguageID'][i][1:]

In [5]:
df

,NaturalLanguageID,ProgrammingLanguageName,RepoID,SourceID,CommentID,comment,label
0,SR,Java,BILD-IT-Advanced-master,src\grupniProjekat_02_03_2016\SearchEngine.java,BILD-IT-Advanced-master/src\grupniProjekat_02_...,Ispisujemo poruku da je drajver loadovan,Functional-Inline
1,SR,Java,BILD-IT-Advanced-master,src\grupniProjekat_02_03_2016\SearchEngine.java,BILD-IT-Advanced-master/src\grupniProjekat_02_...,Ispis država za uneseni kontinent,Functional-Inline
2,SR,Java,BILD-IT-Advanced-master,src\grupniProjekat_02_03_2016\SearchEngine.java,BILD-IT-Advanced-master/src\grupniProjekat_02_...,Slanje upute,Functional-Inline
3,SR,Java,BILD-IT-Advanced-master,src\grupniProjekat_02_03_2016\SearchEngine.java,BILD-IT-Advanced-master/src\grupniProjekat_02_...,Zatvaranje konekcije sa bazom podataka,Functional-Inline
4,SR,Java,BILD-IT-Advanced-master,src\grupniProjekat_HotelManagement\Hotel.java,BILD-IT-Advanced-master/src\grupniProjekat_Hot...,vraca program na pocetak,Functional-Inline
...,...,...,...,...,...,...,...
3371,EN/SR,Java,SCStemmers-master,src\weka\core\stemmers\SCStemmer.java,SCStemmers-master/src\weka\core\stemmers\SCSte...,* Osnovna apstraktna klasa za funkcije zajed...,Functional-Module
3372,SR,Java,SCStemmers-master,src\weka\core\stemmers\SCStemmer.java,SCStemmers-master/src\weka\core\stemmers\SCSte...,* Inicijalizuje pravila za stemovanje * <p>...,Functional-Method
3373,SR,Java,SCStemmers-master,src\weka\core\stemmers\SerbianStemmer.java,SCStemmers-master/src\weka\core\stemmers\Serbi...,* Spisak sufiksnih pravila * <p> * <i>The ...,Functional-Inline
3374,SR,Java,SCStemmers-master,src\weka\core\stemmers\SerbianStemmer.java,SCStemmers-master/src\weka\core\stemmers\Serbi...,* Stemuje reč koja je napisana u dual1 kodir...,Functional-Method


In [6]:
# This is for the case we want to use Functional for all comments

if merge_functional:
    for i in range(df.shape[0]):
        df.loc[i]['label'] = re.sub('-Inline', '', df['label'][i])
        df.loc[i]['label'] = re.sub('-Method', '', df['label'][i])
        df.loc[i]['label'] = re.sub('-Module', '', df['label'][i])


NameError: name 'merge_functional' is not defined

In [7]:
df['label'].value_counts()

Functional-Inline    1946
Functional-Method     967
Functional-Module     315
Code                   70
General                35
Notice                 16
ToDo                   16
IDE                    11
Name: label, dtype: int64

Preprocessing all comments

In [12]:
for i in range(df.shape[0]):
    # pretvaranje celog teksta u mala slova
    if use_lowercase:
        df.loc[i]['comment'] = df.loc[i]['comment'].lower()
    # izbacivanje special character-a
    if remove_special_chars:
        df['comment'][i] = re.sub(r'\W', ' ', df['comment'][i])
        # izbacivanje new line oznake
        df['comment'][i] = re.sub(r'\b[n]\b', '', df['comment'][i])
        # zamena vise razmaka s jednim razmakom
        df['comment'][i] = re.sub(r'\s+', ' ', df['comment'][i], flags=re.I)

In [13]:
for i in range(10):
    print(df['comment'][i])

 ispisujemo poruku da je drajver loadovan 
 ispis država za uneseni kontinent 
 slanje upute 
 zatvaranje konekcije sa bazom podataka 
 vraca program na pocetak 
 paneli za tabove u prozoru admin 
 button za pregled racuna za datog korisnika 
 button za spasavanje update a gosta 
 panel sa boxom i poljem za unos nove sobe na zahtjev gosta 
 polje za unos novog broja sobe 


In [14]:
df.shape

(3376, 7)

## ML 

In [15]:
df['comment'].values

array([' ispisujemo poruku da je drajver loadovan ',
       ' ispis država za uneseni kontinent ', ' slanje upute ', ...,
       ' spisak sufiksnih pravila p i the list of suffix rules i ',
       ' stemuje reč koja je napisana u dual1 kodiranju p i stems a word written in the dual1 coding system i param word reč koju treba stemovati br i the word to be stemmed i return stemovana reč br i the stemmed word i ',
       ' konvertuje zadati string reč ili liniju teksta iz standardnog oblika ćirilice ili latinice u dual1 kodiranje p i converts the given string a word or a line of text from the standard form in the cyrillic or latin script into the dual1 coding system i param wordorline string napisan u standardnom obliku br i a string written in the standard form i return string napisan u dual1 kodiranju br i a string written in the dual1 coding system i '],
      dtype=object)

In [16]:
df.label.value_counts()

Functional-Inline    1946
Functional-Method     967
Functional-Module     315
Code                   70
General                35
Notice                 16
ToDo                   16
IDE                    11
Name: label, dtype: int64

In [54]:
df['label'].values

array(['label', 'Functional-Inline', 'Functional-Inline', ...,
       'Functional-Inline', 'Functional-Inline', 'Functional-Inline'],
      dtype=object)

In [17]:
x = df['comment'].values
y = df['label'].values

In [56]:
# Pokrenuti ovo za koriscenje stemmovanja

import nltk
nltk.download('punkt')

import StemmerByNikola
from StemmerByNikola import stem_str

if use_stemmer:
    stemmed_x = []
    for doc in x:
        stemmed_x.append(stem_str(doc))
    
    x = np.array(stemmed_x)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
['jovi', 'jesam', 'isx', 'u', 'sxkol', '.', 'marij', 'jesam', 'dobr', 'devoj', '.']
 jovi jesam isx u sxkol . marij jesam dobr devoj .


In [19]:
from sklearn.model_selection import StratifiedKFold

kf = StratifiedKFold(n_splits=10, random_state=151, shuffle=True)
kf.get_n_splits(x)

10

In [21]:
import warnings
warnings.filterwarnings('ignore')

In [69]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score# Deo gde se radi obicna vektorizacija bez TF-IDF
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

Logistic regression:
Accuracy: 0.8484322999432978
F1 score: 0.3621583653748154


In [77]:
def create_pipeline(model='log_reg', C=1):
    steps = []
    steps.append(("vect", CountVectorizer(analyzer='word', # lowercase=True, # Menjanjem ovoga testiramo lowercase uticaj
                         ngram_range=(1, n_gram), # Podesavanjem ovoga testiramo bigramski i trigramski uticaj
                         min_df=param_min_df, max_df=param_max_df))) # Podesavanjem ova dva testiracemo uticaj frekvencijskog filtriranja
    if use_tf_idf:
        steps.append(("tfidf", TfidfTransformer(use_idf=use_idf)))
        
    if model == 'log_reg':
        steps.append(("classifier", LogisticRegression(penalty=l_penalty, C=C)))
    elif model == 'bayes':
        steps.append(("classifier", MultinomialNB()))
    elif model == 'bernoulli':
        steps.append(("classifier", BernoulliNB()))
    elif model == 'svm':
        steps.append(("classifier", LinearSVC(penalty=l_penalty, C=C, dual=False)))
        
    return Pipeline(steps)

In [86]:
def cross_validation(x, y, model = 'log_reg'):
    pipeline = create_pipeline(model)

    accuracies = []
    f1_scores = []
#     l_penalty = 'l2'

    for train_index, test_index in kf.split(x, y):
        X_train, X_test = x[train_index], x[test_index]
        y_train, y_test = y[train_index], y[test_index]

        pipeline.fit(X_train, y_train)
        y_pred = pipeline.predict(X_test)

        accuracies.append(accuracy_score(y_test, y_pred))
        f1_scores.append(f1_score(y_test, y_pred, average='macro'))

    print(model)
    print("Accuracy:", sum(accuracies)/len(accuracies))
    print("F1 score:", sum(f1_scores)/len(f1_scores))

In [89]:
def nested_cross_validation(x, y, model='log_reg'):
    accuracies = []
    f1_scores = []
    for train_index, test_index in kf.split(x, y):
        X_train, X_test = x[train_index], x[test_index]
        y_train, y_test = y[train_index], y[test_index]
        parameters = {'C':[0.1, 1, 10, 100, 1000]}

        kf_inner = StratifiedKFold(n_splits=5, random_state=151, shuffle=True)
        kf_inner.get_n_splits(X_train)

        acc_average = []
        f1_average = []
        for c in parameters['C']:
            accuracies_inner = []
            f1_scores_inner = []
            for train_index_inner, test_index_inner in kf_inner.split(X_train, y_train):
                X_train_inner, X_test_inner = X_train[train_index_inner], X_train[test_index_inner]
                y_train_inner, y_test_inner = y_train[train_index_inner], y_train[test_index_inner]

                pipeline_inner = create_pipeline(model, C=c)
                pipeline_inner.fit(X_train_inner, y_train_inner)
                y_pred = pipeline_inner.predict(X_test_inner)

                accuracies_inner.append(accuracy_score(y_test_inner, y_pred))
                f1_scores_inner.append(f1_score(y_test_inner, y_pred, average='macro'))

            acc_average.append(np.mean(accuracies_inner))
            f1_average.append(np.mean(f1_scores_inner))


        pipeline = create_pipeline(model, C=parameters['C'][np.argmax(f1_average)])
        pipeline.fit(X_train, y_train)
        y_pred = pipeline.predict(X_test)
        accuracies.append(accuracy_score(y_test, y_pred))
        f1_scores.append(f1_score(y_test, y_pred, average='macro'))
        print('Accuracy for C = {}: {}'.format(parameters['C'][np.argmax(f1_average)], accuracy_score(y_test, y_pred)))
        print('F1 score for C = {}: {}'.format(parameters['C'][np.argmax(f1_average)], f1_score(y_test, y_pred, average='macro')))

    print(model)
    print("Mean accuracy: ", np.mean(accuracies))
    print("Mean f1 score: ", np.mean(f1_scores))

# Logistic regression L penalty

In [87]:
cross_validation(x, y, 'log_reg')

log_reg
Accuracy: 0.8484322999432978
F1 score: 0.3621583653748154


## Grid search for finding the optimal C value

In [94]:
nested_cross_validation(x, y)

Accuracy for C = 1000: 0.8211143695014663
F1 score for C = 1000: 0.5824740792570446
Accuracy for C = 1000: 0.861764705882353
F1 score for C = 1000: 0.7341134327442318
Accuracy for C = 1000: 0.861764705882353
F1 score for C = 1000: 0.7170624898753298
Accuracy for C = 10: 0.8764705882352941
F1 score for C = 10: 0.561687742810921
Accuracy for C = 1000: 0.861764705882353
F1 score for C = 1000: 0.5213020800575279
Accuracy for C = 1000: 0.8816568047337278
F1 score for C = 1000: 0.5634711779448621
Accuracy for C = 100: 0.8567164179104477
F1 score for C = 100: 0.4134998418296212
Accuracy for C = 1000: 0.9131736526946108
F1 score for C = 1000: 0.8176847733650591
Accuracy for C = 1000: 0.8383233532934131
F1 score for C = 1000: 0.47535723048880946
Accuracy for C = 100: 0.9191616766467066
F1 score for C = 100: 0.5344073662515625
log_reg
Mean accuracy:  0.8691910980662726
Mean f1 score:  0.5921060214624969


## Multinomial Naive Bayes Classifier

In [91]:
cross_validation(x, y, 'bayes')

bayes
Accuracy: 0.8051973317415492
F1 score: 0.3210187579043369


## Bernoulli Naive Bayes Classifier

In [92]:
cross_validation(x, y, 'bernoulli')

bernoulli
Accuracy: 0.759542509808802
F1 score: 0.24040117892364204


## Support Vector Machine

In [93]:
cross_validation(x, y, 'svm')

svm
Accuracy: 0.8872344630716068
F1 score: 0.6066933802213528


In [ ]:
nested_cross_validation(x, y, 'svm')

Accuracy for C = 1: 0.8475073313782991
F1 score for C = 1: 0.6102354566337617
Accuracy for C = 10: 0.8735294117647059
F1 score for C = 10: 0.729168797294754
Accuracy for C = 1: 0.8823529411764706
F1 score for C = 1: 0.7154070113809112
Accuracy for C = 1: 0.8823529411764706
F1 score for C = 1: 0.6130462659626968
Accuracy for C = 10: 0.8676470588235294
F1 score for C = 10: 0.5248611556590617
Accuracy for C = 1: 0.893491124260355
F1 score for C = 1: 0.5768283410424062
Accuracy for C = 1: 0.8656716417910447
F1 score for C = 1: 0.4530805157820453
Accuracy for C = 10: 0.9221556886227545
F1 score for C = 10: 0.8079014046952187
Accuracy for C = 10: 0.844311377245509
F1 score for C = 10: 0.4737452603755417


In [71]:
t_end = time()
print(t_end - t_start)

157.20130968093872
